# Task: update ObservationLog using information in "contact.xlsx"

In [1]:
import numpy as np
import pandas as pd
import os
from datetime import datetime

In [2]:
def checkAndAssign(df: pd.DataFrame, contact_list: pd.DataFrame, checking_by: str = '氏名'):
    """
    Assign contact information from contact list to observationLog file
    
    Input:
        df (pd.DataFrame): data frame to be updated
        contact_list (pd.DataFrame): contact information
        checking_by (str): criteria to assign infomation, default name
    
    output:
        df (pd.DataFrame): updated df
        no_all (np.array): list of users that have no information from contact list
    """
    no_name = []
    no_aff = []
    no_title = []
    for i,name in enumerate(df['PI-name']):
        if (name in np.array(contact_list.iloc[:,0])) or (name in np.array(contact_list.iloc[:,1])):
            # assign email address: at col 7
            df.iloc[i,7] = str(np.array(contact_list[np.array(contact_list[checking_by])==name])[0][-1])
        elif name not in no_name:
            # if name not recorded, put into no_name list
            no_name = np.append(no_name,name)
            #print(df.iloc[i,3:5])
            aff, title = df.iloc[i,3:5]
            no_aff = np.append(no_aff,aff)
            no_title = np.append(no_title,title)
        else:
            continue
    no_all = np.column_stack((no_name,no_aff,no_title))
    return df, no_all

def createNoNameDF(no_name_list: np.array):
    """
    Convert the list of no contact information to pd.DataFrame
    
    Input:
        no_name_list (np.array): a matrix 
    
    Output:
        df (pd.DataFrame): data frame with column names
    """
    # make a matrix with no names
    #dt = np.column_stack((no_name_list,nas))
    df = pd.DataFrame(no_name_list, columns=['氏名','所属','職名'])
    return df


def combineContactList(contact_list1: str, contact_list2: str):
    """
    Combine two contact lists (organized by Python and by hands)
    
    Input:
        contact_list1 (str): filename of list 1
        contact_list2 (str): filename of list 2
    Output;
        output (pd.DataFrame): data frame combining two lists
    """
    cl1 = pd.read_excel(contact_list1)
    cl2 = pd.read_excel(contact_list2)
    #output = np.row_stack((cl1,cl2))
    # concat according to column names
    output = pd.concat([cl1,cl2], axis=0)
    print(f"Two contact lists shape: {cl1.shape} && {cl2.shape}")
    return output

In [3]:
############### Configuration: path, folder, date ################
today = datetime.today().date()
print(f"Today is {today}")
# define save filename by date
out_fn = f"ObservationLog_daily_{today}.xlsx"
# specify file to read, and check its column names
path = "./obslog/"
obs_fn = path + "ObservationLog_daily.xlsx"
##################################################################

Today is 2023-12-18


In [4]:
# reading observationLog into df
df = pd.read_excel(obs_fn)
print(df.columns)

# trying read contact_list for checking in later part
contact_fn = "contact_list.xlsx"
contact_list = pd.read_excel(contact_fn)
print(contact_list.columns)

Index(['yyyy/mm/dd', 'Ship-name', 'PI-name', 'Pl-affiliation', 'PI-position',
       'Instruments', 'CTD (0/1)', 'PI-email', 'Unnamed: 8', 'Ship-name list'],
      dtype='object')
Index(['Unnamed: 0', '氏名', '所属', '職名', 'メールアドレス'], dtype='object')


In [5]:
# read two contact lists: 1) organized from 共同利用研究者連絡先（公募利用） 2) manually collected
contact1 = "contact_list.xlsx"
contact2 = "contact_list_noName.xlsx"
all_contact = combineContactList(contact_list1=contact1, contact_list2=contact2)
print(all_contact.shape)

Two contact lists shape: (92, 5) && (8, 5)
(100, 5)


In [6]:
# assign email address to new df
#new_df,no_exist_name = checkAndAssign(df,contact_list)
new_df,no_exist_name = checkAndAssign(df,all_contact)
####################################################
#print(new_df)
#print(no_exist_name)

c:\Users\Arthurli\anaconda3\lib\site-packages\ipykernel_launcher.py:7: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  import sys


In [7]:
# save no_exist_name to an excel file for further processing
nen_df = createNoNameDF(no_exist_name)
nen_df.to_excel(path+"no_name_list.xlsx")

In [8]:
# save as temporary excel file -> copy and update on drive
new_df.to_excel(path+out_fn)

End of file